In [1]:
import numpy as np

In [2]:
import torch
import pandas as pd
# import nltk
import os

In [3]:
from preprocessing import clean_text

In [4]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_word = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muz94\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
test_data = pd.read_csv("test.csv", dtype={"test_id": int, "question1": str, "question2": str})
train_data = pd.read_csv("train/train.csv", dtype={'id': int, 'qid1':int, 'qid2': int, 'question1':str, 'question2':str, 'is_duplicate': int})
test_data.set_index('test_id', inplace = True)
train_data.set_index('id', inplace = True)

In [6]:
data = train_data.sample(n=100000).copy()
data.loc[data['is_duplicate']==1]

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
269379,38358,96545,What is your favourite book and why?,What is your favourite book?,1
355210,484421,484422,How can you make a creative Humpty Dumpty cost...,What are some ways to make a Humpty Dumpty cos...,1
196799,297589,297590,What does it mean if a dog has diarrhea with b...,What does it mean if your dog is suffering fro...,1
159666,5059,55281,How can the drive from Edmonton to Auckland be...,How can the drive from Edmonton to Auckland be...,1
271550,75366,169831,Why should I study physics?,Why do we study physics?,1
...,...,...,...,...,...
71003,9903,122264,What should I do to get into an Ivy League col...,How can I get into an Ivy League college?,1
98184,163200,163201,What is the skeletal system?,What is the skeletal system and its function?,1
142991,107441,226656,Which book shall I prefer for the CAT?,Which book should I prefer for the CAT?,1


In [7]:
def sen2list(sentence: str):
    mod_lst = []
    try:
        sentence = clean_text(sentence)

        lst = word_tokenize(sentence)
        
        #ps = PorterStemmer()
        for word in lst:
            c = word[0]
            if word != "" and (("a"<=c and c <="z") or ("A" <= c and c <="Z") or ("0"<= c and c <="9")):
                #mod_word = ps.stem(word)
                mod_word = word
                if mod_word not in stop_word:
                    mod_lst.append(mod_word)
        
    except:
        print(sentence)
    
    return mod_lst

In [8]:
sen2list(data['question1'].iloc[0])

['favourite', 'book']

In [9]:
data['lst1'] = data.apply(lambda row: sen2list(row.question1), axis = 1)
data['lst2'] = data.apply(lambda row: sen2list(row.question2), axis = 1)

In [12]:
data

,qid1,qid2,question1,question2,is_duplicate,lst1,lst2
id,,,,,,,
269379,38358,96545,What is your favourite book and why?,What is your favourite book?,1,"[favourite, book]","[favourite, book]"
255652,180850,370585,Who will win the 25th match between Australia ...,Who will win the 3rd match between England and...,0,"[win, 25th, match, australia, bangladesh, 2016...","[win, 3rd, match, england, bangladesh, 2016, k..."
57252,100630,100631,What is the chemical equation for iron and sul...,What is the chemical equation for calcium and ...,0,"[chemical, equation, iron, sulphuric, acid]","[chemical, equation, calcium, water]"
376416,507555,507556,What is the formula for centroid of a cube?,What is the centroid of a cube?,0,"[formula, centroid, cube]","[centroid, cube]"
355210,484421,484422,How can you make a creative Humpty Dumpty cost...,What are some ways to make a Humpty Dumpty cos...,1,"[make, creative, humpty, dumpty, costume]","[ways, make, humpty, dumpty, costume]"
...,...,...,...,...,...,...,...
399342,532596,532597,Why does the USA have a very poor public trans...,"What is the meaning of this thought,""A develop...",0,"[america, poor, public, transport, system, com...","[meaning, thought, developed, country, place, ..."
221852,329288,329289,How can I mention someone so that all their fo...,How can I see @mentions for another Twitter user?,0,"[mention, someone, followers, see, message]","[see, mentions, another, twitter, user]"
148570,234220,234221,What are some good ways to explain to a teache...,How do I prove my teacher wrong?,1,"[good, ways, explain, teacher, wrong]","[prove, teacher, wrong]"


In [11]:
data = data.loc[(data['lst1'].str.len() > 0) & (data['lst2'].str.len() > 0),:].copy()

#### Corpus

In [13]:
from gensim import corpora

In [14]:
corpus = data['lst1'].tolist() + data['lst2'].tolist()
len(corpus)

199934

In [15]:
dictionary = corpora.Dictionary(corpus)
len(dictionary.token2id)

47548

In [16]:
seq_len = 50

In [17]:
def text_to_seq(lst: list) -> list:
    if len(lst) < seq_len:
        seq = [0 for n in range(seq_len-len(lst))] + [dictionary.token2id[word]+1 for word in lst]
    else:
        seq = [dictionary.token2id[word]+1 for word in lst[-seq_len:]]
    return seq

In [18]:
data['seq1'] = data.apply(lambda row: text_to_seq(row.lst1), axis = 1)
data['seq2'] = data.apply(lambda row: text_to_seq(row.lst2), axis = 1)

In [19]:
data

,qid1,qid2,question1,question2,is_duplicate,lst1,lst2,seq1,seq2
id,,,,,,,,,
269379,38358,96545,What is your favourite book and why?,What is your favourite book?,1,"[favourite, book]","[favourite, book]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
255652,180850,370585,Who will win the 25th match between Australia ...,Who will win the 3rd match between England and...,0,"[win, 25th, match, australia, bangladesh, 2016...","[win, 3rd, match, england, bangladesh, 2016, k...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
57252,100630,100631,What is the chemical equation for iron and sul...,What is the chemical equation for calcium and ...,0,"[chemical, equation, iron, sulphuric, acid]","[chemical, equation, calcium, water]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
376416,507555,507556,What is the formula for centroid of a cube?,What is the centroid of a cube?,0,"[formula, centroid, cube]","[centroid, cube]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
355210,484421,484422,How can you make a creative Humpty Dumpty cost...,What are some ways to make a Humpty Dumpty cos...,1,"[make, creative, humpty, dumpty, costume]","[ways, make, humpty, dumpty, costume]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
399342,532596,532597,Why does the USA have a very poor public trans...,"What is the meaning of this thought,""A develop...",0,"[america, poor, public, transport, system, com...","[meaning, thought, developed, country, place, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
221852,329288,329289,How can I mention someone so that all their fo...,How can I see @mentions for another Twitter user?,0,"[mention, someone, followers, see, message]","[see, mentions, another, twitter, user]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
148570,234220,234221,What are some good ways to explain to a teache...,How do I prove my teacher wrong?,1,"[good, ways, explain, teacher, wrong]","[prove, teacher, wrong]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
data['seq1'].iloc[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1]

### Split set

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(data[["seq1", "seq2"]], data["is_duplicate"], test_size=0.2)

In [22]:
X_train

,seq1,seq2
id,,
327428,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
360128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
198116,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
266558,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
155916,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
202801,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
240086,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
25438,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [23]:
y_train

id
327428    0
360128    0
198116    1
266558    0
155916    0
         ..
202801    0
240086    0
25438     0
36875     0
123872    0
Name: is_duplicate, Length: 79973, dtype: int32

In [153]:
type(X_train['seq1'].iloc[0])

list

# Import Glove

In [24]:
EMBEDDING_FILE = 'glove.6B.100d.txt'

In [25]:
embed_dict = {}
for o in open(EMBEDDING_FILE, encoding='utf-8'):
    o_lst = o.split(" ")
    embed_dict[o_lst[0]] = np.asarray(o_lst[1:], dtype='float32')[:100]

In [26]:
embed_dict['the'].shape

(100,)

In [27]:
len(embed_dict.keys())

400001

In [28]:
embed_dict['followed']

array([ 5.2346e-02, -6.4872e-02, -2.2969e-01, -1.9626e-02,  2.4551e-01,
        2.6228e-01,  4.5854e-01,  2.2948e-01, -4.9086e-01, -2.0671e-01,
        3.4402e-01, -4.6660e-02,  7.4588e-02,  4.5667e-01,  3.3393e-01,
        1.2274e-01, -4.3574e-01, -6.4115e-02, -5.8127e-01,  6.0061e-02,
        8.0394e-01, -4.8664e-01, -8.5516e-03,  7.4187e-01,  3.9295e-01,
       -2.0148e-01, -2.1781e-01,  5.5098e-02,  1.2245e-01, -1.5723e-01,
       -2.0093e-01,  2.1990e-01,  1.2953e-01,  1.0737e-02, -3.0524e-01,
        2.2466e-01,  3.8637e-01, -1.6129e-01, -4.1141e-01,  5.2801e-02,
       -4.4697e-01, -2.8872e-01,  1.7551e-01,  2.1389e-01,  1.5470e-01,
       -3.3769e-01,  3.8086e-01, -3.3577e-01,  4.8071e-01, -8.7916e-01,
        6.3989e-02,  7.5071e-02,  1.1240e-01,  7.3637e-01, -1.6774e-01,
       -2.3195e+00, -5.0312e-01,  9.5163e-02,  1.1419e+00,  7.6243e-01,
       -3.8345e-01,  4.9045e-01,  2.3280e-01, -8.0937e-02,  5.3929e-01,
       -1.3643e-01, -2.2535e-01, -1.4808e-01,  2.1617e-01,  1.90

In [29]:
special = [w for w in embed_dict.keys() if not (("a"<=w[0] and w[0] <="z") or ("A" <= w[0] and w[0] <="Z") or ("0"<= w[0] and w[0] <="9"))]

In [30]:
special[1000]

'ས'

In [31]:
all_embs = np.stack(list(embed_dict.values()))

In [32]:
all_embs.shape

(400001, 100)

In [33]:
vocab_size = len(dictionary.token2id) + 1
substi_vector = np.asarray([0 for n in range(100)], dtype = 'float32')
      
# Adding again 1 because of reserved 0 index
subs_num = 1
embedding_matrix = np.zeros((vocab_size,100))
ps = PorterStemmer()
for word, idx in dictionary.token2id.items():
    if word in embed_dict.keys():
        embedding_matrix[idx+1] = embed_dict[word]
    else:
        mod_word = ps.stem(word)
        if mod_word in embed_dict.keys():
            embedding_matrix[idx+1] = embed_dict[mod_word]
        else:
            embedding_matrix[idx+1] = substi_vector
            subs_num += 1
        
embedding_matrix[0] = substi_vector

In [34]:
subs_num

9170

In [35]:
embedding_matrix.shape

(47549, 100)

In [36]:
embedding_matrix.dtype

dtype('float64')

In [37]:
embedding_matrix[1]

array([-1.97439998e-01,  4.48309988e-01,  1.36889994e-01, -1.55949995e-01,
        9.35999990e-01,  7.29860008e-01,  3.40990007e-01, -3.38959992e-01,
       -8.95690024e-02, -4.77059990e-01,  3.51119995e-01, -4.21979994e-01,
       -1.22210003e-01, -6.33750036e-02, -4.58200008e-01,  7.87230015e-01,
        9.40450013e-01,  8.11010003e-02, -2.32240006e-01,  4.07779992e-01,
        3.32580000e-01, -4.44579989e-01, -4.71170008e-01,  1.48519993e-01,
        9.63079989e-01, -6.52669966e-02, -5.36610000e-02, -6.74740016e-01,
       -4.23640013e-01,  9.43920016e-02, -3.86680007e-01,  1.82370007e-01,
       -1.28460005e-01, -2.19520003e-01, -5.89929998e-01,  7.36020029e-01,
       -2.40089998e-01,  3.23920012e-01, -2.46629998e-01, -4.06839997e-01,
       -5.24680018e-01,  4.61739987e-01, -1.49360001e-01, -1.19989999e-01,
       -1.39899999e-01, -4.49440002e-01, -2.65650004e-01, -7.00609982e-01,
        3.01880002e-01, -1.12089999e-01,  6.63230002e-01,  3.96979988e-01,
        6.91579998e-01,  

# Create DataLoader

In [38]:
X_train_1 = [np.asarray(lst, dtype='long') for lst in X_train['seq1'].values]
X_train_1 = np.stack(X_train_1)
X_train_1.shape

(79973, 50)

In [39]:
X_train_1[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 7882, 1780])

In [40]:
X_train_2 = [np.asarray(lst, dtype='long') for lst in X_train['seq2'].values]
X_train_2 = np.stack(X_train_2)
X_train_2.shape

(79973, 50)

In [41]:
X_train_2[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 5915])

In [42]:
import torch

In [43]:
X_train_1

array([[    0,     0,     0, ...,     0,  7882,  1780],
       [    0,     0,     0, ...,     0,     0,   785],
       [    0,     0,     0, ...,    92, 13550,  4389],
       ...,
       [    0,     0,     0, ...,   568,    79,   845],
       [    0,     0,     0, ...,  2792,  1027,  1290],
       [    0,     0,     0, ...,  1178,   121,  1177]])

In [44]:
X_train_1_tensor = torch.tensor(X_train_1, dtype=torch.long)

In [45]:
X_train_1_tensor

tensor([[    0,     0,     0,  ...,     0,  7882,  1780],
        [    0,     0,     0,  ...,     0,     0,   785],
        [    0,     0,     0,  ...,    92, 13550,  4389],
        ...,
        [    0,     0,     0,  ...,   568,    79,   845],
        [    0,     0,     0,  ...,  2792,  1027,  1290],
        [    0,     0,     0,  ...,  1178,   121,  1177]])

In [46]:
X_train_2_tensor = torch.tensor(X_train_2, dtype=torch.long)

In [47]:
X_train_2_tensor

tensor([[    0,     0,     0,  ...,     0,     0,  5915],
        [    0,     0,     0,  ...,     0,   785,  3342],
        [    0,     0,     0,  ...,    92, 13550, 11034],
        ...,
        [    0,     0,     0,  ...,   568,   195,   845],
        [    0,     0,     0,  ...,   458,   458,   357],
        [    0,     0,     0,  ...,   726,  1347,   326]])

In [48]:
y_train =  [np.asarray(lst, dtype='float32') for lst in y_train.values]

In [49]:
y_train = np.stack(y_train)
y_train

array([0., 0., 1., ..., 0., 0., 0.], dtype=float32)

In [50]:
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

In [51]:
y_train_tensor

tensor([0., 0., 1.,  ..., 0., 0., 0.])

In [52]:
X_validation_1 = [np.asarray(lst, dtype='long') for lst in X_validation['seq1'].values]
X_validation_1 = np.stack(X_validation_1)
X_validation_1.shape

(19994, 50)

In [53]:
X_validation_2 = [np.asarray(lst, dtype='long') for lst in X_validation['seq2'].values]
X_validation_2 = np.stack(X_validation_2)
X_validation_2.shape

(19994, 50)

In [54]:
y_validation =  [np.asarray(lst, dtype='float32') for lst in y_validation.values]
y_validation = np.stack(y_validation)
y_validation

array([1., 0., 0., ..., 0., 0., 1.], dtype=float32)

In [55]:
X_validation_1_tensor = torch.tensor(X_validation_1, dtype=torch.long)
X_validation_2_tensor = torch.tensor(X_validation_2, dtype=torch.long)
y_validation_tensor = torch.tensor(y_validation, dtype=torch.float32)

In [56]:
train = torch.utils.data.TensorDataset(X_train_1_tensor, X_train_2_tensor, y_train_tensor)

In [57]:
train.__getitem__(0)

(tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         7882, 1780]),
 tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 5915]),
 tensor(0.))

In [58]:
validation = torch.utils.data.TensorDataset(X_validation_1_tensor, X_validation_2_tensor, y_validation_tensor)

In [59]:
validation.__len__()

19994

In [64]:
import multiprocessing

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
batch_size = 32

#core_number = multiprocessing.cpu_count()

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train,  # The training samples.
            shuffle=True, # Select batches randomly
            batch_size = batch_size, # Trains with this batch size.
            num_workers = 1
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            validation, # The validation samples.
            shuffle=False, # Pull out batches sequentially.
            batch_size = batch_size, # Evaluate with this batch size.
            num_workers = 1
        )

In [65]:
train_dataloader.__len__()

2500

In [68]:
n = 0
for batch in train_dataloader:
    if n == 0:
        batch_sample = batch
        break

In [69]:
batch_sample

[tensor([[    0,     0,     0,  ..., 36068,  5096, 11495],
         [    0,     0,     0,  ...,  8991,   656,  8990],
         [    0,     0,     0,  ...,  6524,   373, 11206],
         ...,
         [    0,     0,     0,  ...,   213,   838,  1118],
         [    0,     0,     0,  ...,     0,   230,  1904],
         [    0,     0,     0,  ...,     0,    92,  2879]]),
 tensor([[    0,     0,     0,  ...,  1750,   107, 24112],
         [    0,     0,     0,  ...,     0,  7320,  8991],
         [    0,     0,     0,  ...,  6524,   373, 37783],
         ...,
         [    0,     0,     0,  ...,  4132,  6608, 37692],
         [    0,     0,     0,  ...,     0,   230,  1893],
         [    0,     0,     0,  ...,  2879,  1957,   311]]),
 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
         1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.])]

### Test

In [85]:
test_embedding = nn.Embedding(embedding_dim, 100)
test_embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32)) 

In [86]:
embeds1 = test_embedding(batch_sample[0])
embeds2 = test_embedding(batch_sample[1])

In [87]:
(embeds1.shape, embeds2.shape)

(torch.Size([32, 50, 100]), torch.Size([32, 50, 100]))

In [79]:
test_lstm1 = nn.LSTM(input_size = 100, hidden_size = hidden_dim, batch_first = True)
test_lstm2 = nn.LSTM(input_size = 100, hidden_size = hidden_dim, batch_first = True)

In [80]:
lstm_out1, hc1 = test_lstm1(embeds1)

In [81]:
lstm_out2, _ = test_lstm2(embeds2)

In [82]:
(lstm_out1.shape, lstm_out2.shape)

(torch.Size([32, 50, 50]), torch.Size([32, 50, 50]))

In [83]:
lstm_out1.size(0)

32

In [84]:
lstm_out2

tensor([[[ 1.1695e-02,  2.8581e-02, -5.8257e-02,  ..., -1.0807e-02,
           2.4341e-02, -4.1429e-02],
         [ 1.8354e-02,  4.3031e-02, -9.1091e-02,  ..., -1.7150e-02,
           4.1275e-02, -5.5303e-02],
         [ 2.2084e-02,  5.0921e-02, -1.0920e-01,  ..., -2.1384e-02,
           5.2224e-02, -6.0224e-02],
         ...,
         [ 7.5206e-02, -2.6757e-02, -1.7488e-01,  ..., -7.7930e-02,
           1.1781e-01, -1.5541e-01],
         [ 5.3420e-02, -9.4440e-03, -1.0597e-01,  ..., -5.4733e-02,
           2.6130e-03, -1.6026e-01],
         [-5.3312e-03,  1.2484e-01, -5.0928e-02,  ..., -9.3307e-03,
          -4.3834e-03, -2.1225e-02]],

        [[ 1.1695e-02,  2.8581e-02, -5.8257e-02,  ..., -1.0807e-02,
           2.4341e-02, -4.1429e-02],
         [ 1.8354e-02,  4.3031e-02, -9.1091e-02,  ..., -1.7150e-02,
           4.1275e-02, -5.5303e-02],
         [ 2.2084e-02,  5.0921e-02, -1.0920e-01,  ..., -2.1384e-02,
           5.2224e-02, -6.0224e-02],
         ...,
         [ 1.5376e-01, -3

#### Match layer

In [90]:
lstm_out1_batch1 = lstm_out1[0:1,:,:]

In [91]:
lstm_out1_batch1

tensor([[[ 0.0159,  0.0629,  0.0524,  ...,  0.1008, -0.2170, -0.0282],
         [-0.0475,  0.1232,  0.0289,  ...,  0.1984, -0.2090, -0.1479],
         [-0.0531,  0.1867,  0.0486,  ..., -0.0209, -0.1780, -0.2003],
         ...,
         [ 0.0247, -0.0087,  0.0237,  ...,  0.0852, -0.0313,  0.0227],
         [ 0.0247, -0.0087,  0.0237,  ...,  0.0852, -0.0313,  0.0227],
         [ 0.0247, -0.0087,  0.0237,  ...,  0.0852, -0.0313,  0.0227]]],
       grad_fn=<SliceBackward>)

In [92]:
lstm_out2_batch1_end = lstm_out2[0:1,-1, :]

In [93]:
lstm_out2_batch1_end

tensor([[-0.0505,  0.0447,  0.0301, -0.0183,  0.0156, -0.0491,  0.0294, -0.0456,
         -0.0847, -0.0942,  0.0158, -0.0474,  0.0219,  0.0586,  0.0160, -0.0663,
         -0.0837,  0.0236,  0.0309,  0.0120,  0.0744, -0.0974,  0.0560, -0.0772,
         -0.0678,  0.1215,  0.1064, -0.0748, -0.0248,  0.0711, -0.0515, -0.0497,
          0.0197, -0.0126, -0.0042, -0.0658, -0.0394,  0.0057,  0.0574, -0.0740,
         -0.0131, -0.0185, -0.0333,  0.0159,  0.0171,  0.0015, -0.1250, -0.0598,
          0.0842, -0.1179]], grad_fn=<SliceBackward>)

In [95]:
lstm_out1_batch1 = lstm_out1_batch1.view(50, 50)
lstm_out1_batch1

tensor([[ 0.0159,  0.0629,  0.0524,  ...,  0.1008, -0.2170, -0.0282],
        [-0.0475,  0.1232,  0.0289,  ...,  0.1984, -0.2090, -0.1479],
        [-0.0531,  0.1867,  0.0486,  ..., -0.0209, -0.1780, -0.2003],
        ...,
        [ 0.0247, -0.0087,  0.0237,  ...,  0.0852, -0.0313,  0.0227],
        [ 0.0247, -0.0087,  0.0237,  ...,  0.0852, -0.0313,  0.0227],
        [ 0.0247, -0.0087,  0.0237,  ...,  0.0852, -0.0313,  0.0227]],
       grad_fn=<ViewBackward>)

In [97]:
lstm_out1_batch1.size(0)

50

In [90]:
test_para = nn.Parameter(torch.rand(30, 50))

In [91]:
test_para.size()

torch.Size([30, 50])

In [92]:
test_para

Parameter containing:
tensor([[0.7368, 0.9743, 0.9150,  ..., 0.4761, 0.7038, 0.7414],
        [0.6111, 0.2289, 0.7476,  ..., 0.6743, 0.9194, 0.1988],
        [0.1597, 0.7428, 0.6512,  ..., 0.3281, 0.3146, 0.8502],
        ...,
        [0.0898, 0.8254, 0.2690,  ..., 0.8089, 0.3538, 0.1819],
        [0.6433, 0.8582, 0.3460,  ..., 0.8198, 0.5527, 0.6052],
        [0.0500, 0.1903, 0.7117,  ..., 0.3004, 0.5919, 0.3825]],
       requires_grad=True)

In [108]:
input1 = torch.randn(10, 128)
input2 = torch.randn(10, 128)
output = F.cosine_similarity(input1, input2)
print(output)

tensor([ 0.0812,  0.1135,  0.0650,  0.0905,  0.1343, -0.0729,  0.2088, -0.1368,
         0.0516, -0.1185])


In [85]:
input1 = torch.randn(5, 128)
input2 = torch.randn(5, 128)
test_cos = nn.CosineSimilarity(dim=1)
output = test_cos(input1, input2)

In [86]:
output

tensor([-0.0089, -0.1298, -0.0036, -0.0451,  0.1058])

In [108]:
seq = 0
batch1_t = lstm_out1_batch1[seq,:]
batch2_t = lstm_out2_batch1_end[0,:]

In [109]:
(batch1_t, batch2_t)

(tensor([ 0.0159,  0.0629,  0.0524, -0.0969,  0.1361, -0.0815,  0.1097, -0.1691,
          0.1185,  0.1331,  0.0373,  0.0901, -0.0055,  0.0335,  0.0008,  0.0258,
          0.0282,  0.0816,  0.2127,  0.0889, -0.1542, -0.0449,  0.2488,  0.1001,
         -0.1428, -0.0919,  0.1175,  0.0391,  0.1426, -0.0382, -0.0996,  0.1259,
         -0.0713,  0.0221, -0.1125, -0.0523, -0.1694, -0.0932,  0.1069, -0.1702,
          0.0900,  0.1949, -0.0317, -0.0439,  0.0504,  0.0022, -0.0338,  0.1008,
         -0.2170, -0.0282], grad_fn=<SliceBackward>),
 tensor([-0.0505,  0.0447,  0.0301, -0.0183,  0.0156, -0.0491,  0.0294, -0.0456,
         -0.0847, -0.0942,  0.0158, -0.0474,  0.0219,  0.0586,  0.0160, -0.0663,
         -0.0837,  0.0236,  0.0309,  0.0120,  0.0744, -0.0974,  0.0560, -0.0772,
         -0.0678,  0.1215,  0.1064, -0.0748, -0.0248,  0.0711, -0.0515, -0.0497,
          0.0197, -0.0126, -0.0042, -0.0658, -0.0394,  0.0057,  0.0574, -0.0740,
         -0.0131, -0.0185, -0.0333,  0.0159,  0.0171,  

In [110]:
mod_batch1_t = torch.mul(test_para, batch1_t)
mod_batch2_t = torch.mul(test_para, batch2_t)

In [111]:
(mod_batch1_t, mod_batch2_t)

(tensor([[ 0.0094,  0.0082,  0.0403,  ...,  0.0994, -0.0723, -0.0077],
         [ 0.0059,  0.0449,  0.0193,  ...,  0.0750, -0.1023, -0.0256],
         [ 0.0129,  0.0460,  0.0019,  ...,  0.0398, -0.1424, -0.0098],
         ...,
         [ 0.0134,  0.0622,  0.0284,  ...,  0.0262, -0.0581, -0.0111],
         [ 0.0040,  0.0247,  0.0087,  ...,  0.0547, -0.1446, -0.0198],
         [ 0.0065,  0.0379,  0.0398,  ...,  0.0770, -0.0224, -0.0133]],
        grad_fn=<MulBackward0>),
 tensor([[-0.0297,  0.0058,  0.0231,  ..., -0.0589,  0.0281, -0.0320],
         [-0.0189,  0.0319,  0.0111,  ..., -0.0445,  0.0397, -0.1069],
         [-0.0410,  0.0327,  0.0011,  ..., -0.0236,  0.0553, -0.0408],
         ...,
         [-0.0427,  0.0442,  0.0163,  ..., -0.0155,  0.0225, -0.0461],
         [-0.0128,  0.0175,  0.0050,  ..., -0.0324,  0.0561, -0.0828],
         [-0.0207,  0.0269,  0.0229,  ..., -0.0456,  0.0087, -0.0554]],
        grad_fn=<MulBackward0>))

In [112]:
match_out_t=test_cos(mod_batch1_t, mod_batch2_t)

In [123]:
match_out_t

tensor([ 0.0331, -0.0832,  0.2572,  0.1136,  0.0721,  0.2472,  0.3161,  0.1267,
         0.1933,  0.0192,  0.3663,  0.0510, -0.0863,  0.2568,  0.2178,  0.1372,
        -0.0304, -0.1672,  0.0943,  0.0518,  0.0653,  0.0727, -0.0527, -0.0521,
        -0.1651, -0.1871,  0.0469,  0.2182,  0.0892, -0.0248],
       grad_fn=<DivBackward0>)

In [87]:
match_out_lst = []
for seq in range(lstm_out1_batch1.size(0)):
    batch1_t = lstm_out1_batch1[seq,:]
    batch2_t = lstm_out2_batch1_end[0,:]
    mod_batch1_t = torch.mul(test_para, batch1_t)
    mod_batch2_t = torch.mul(test_para, batch2_t)
    match_out_t = test_cos(mod_batch1_t, mod_batch2_t)
    match_out_lst.append(match_out_t)
match_out = torch.stack(match_out_lst)

NameError: name 'lstm_out1_batch1' is not defined

In [122]:
match_out

tensor([[ 0.1138, -0.0389,  0.1248,  ...,  0.0591,  0.1998, -0.0669],
        [ 0.1146,  0.0374,  0.0978,  ...,  0.0980,  0.2666,  0.1044],
        [ 0.1106,  0.0962,  0.0490,  ..., -0.0353,  0.2991,  0.0340],
        ...,
        [ 0.0331, -0.0832,  0.2572,  ...,  0.2182,  0.0892, -0.0248],
        [ 0.0331, -0.0832,  0.2572,  ...,  0.2182,  0.0892, -0.0248],
        [ 0.0331, -0.0832,  0.2572,  ...,  0.2182,  0.0892, -0.0248]],
       grad_fn=<StackBackward>)

In [124]:
match_out.size()

torch.Size([50, 30])

In [125]:
T1 = torch.Tensor([[3,2],[1,2]])

# Create a 1-D tensor
T2 = torch.Tensor([[3,4],[10, 8]])

In [126]:
torch.stack([T1, T2], dim = 0 )

tensor([[[ 3.,  2.],
         [ 1.,  2.]],

        [[ 3.,  4.],
         [10.,  8.]]])

In [113]:
test_repeat = torch.tensor([1, 2, 3])
test_repeat.repeat(4, 1)
#test_repeat

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

In [93]:
test_cos = nn.CosineSimilarity(dim=1)

In [94]:
def match_cal(lstm_out1, lstm_out2, test_para):
    match_out_all = []
    for b in range(lstm_out1.size(0)):
        match_out_lst = []
        lstm_out1_batch1 = lstm_out1[b,:,:]
        lstm_out2_batch1_end = lstm_out2[b, -1, :]
        for seq in range(lstm_out1_batch1.size(0)):
            batch1_t = lstm_out1_batch1[seq,:]
            batch2_t = lstm_out2_batch1_end
            mod_batch1_t = torch.mul(test_para, batch1_t)
            mod_batch2_t = torch.mul(test_para, batch2_t)
            match_out_t = test_cos(mod_batch1_t, mod_batch2_t)
            match_out_lst.append(match_out_t)
        match_out = torch.stack(match_out_lst)
        match_out_all.append(match_out)
    return torch.stack(match_out_all)

In [95]:
match_1 = match_cal(lstm_out1, lstm_out2, test_para)

In [96]:
match_1

tensor([[[-2.0313e-01, -3.6478e-02, -1.5178e-01,  ..., -3.2423e-03,
          -5.7971e-02, -1.9816e-01],
         [-2.3384e-01, -7.8140e-02, -1.8306e-01,  ..., -1.4328e-02,
          -7.3213e-02, -2.1912e-01],
         [-2.5710e-01, -1.1298e-01, -2.0583e-01,  ..., -2.7429e-02,
          -8.5672e-02, -2.4001e-01],
         ...,
         [ 8.5128e-02,  1.5229e-01,  1.1737e-01,  ..., -1.0582e-01,
           8.5553e-04,  1.0023e-01],
         [ 1.6592e-01,  1.2343e-01,  2.7464e-01,  ...,  9.0229e-02,
           1.2870e-01,  1.2398e-01],
         [-7.6293e-02,  3.4264e-01,  1.1801e-01,  ...,  1.4987e-01,
          -7.4881e-02, -3.8145e-02]],

        [[-7.7823e-03,  2.5012e-01,  1.1933e-01,  ...,  2.2397e-01,
           9.2251e-02, -1.4692e-01],
         [-3.1076e-02,  2.2356e-01,  1.0798e-01,  ...,  2.2122e-01,
           7.4549e-02, -1.6022e-01],
         [-5.1704e-02,  1.9971e-01,  9.9267e-02,  ...,  2.1516e-01,
           5.8214e-02, -1.7283e-01],
         ...,
         [ 5.0262e-03,  8

In [97]:
match_2 = match_cal(lstm_out2, lstm_out1, test_para)

In [98]:
match_2.size()

torch.Size([32, 50, 30])

New lstm

In [99]:
test_lstm_new1 = nn.LSTM(input_size = 30, hidden_size = hidden_dim2, batch_first=True)
test_lstm_new2 = nn.LSTM(input_size = 30, hidden_size = hidden_dim2, batch_first=True)

In [100]:
_,(lstm_new_out1,_) = test_lstm_new1(match_1)

In [101]:
lstm_new_out1 = lstm_new_out1.view(lstm_new_out1.size(1), lstm_new_out1.size(2))

In [102]:
lstm_new_out1.size()

torch.Size([32, 10])

In [106]:
lstm_new_out1

tensor([[ 8.0518e-02, -1.4444e-01,  5.3251e-02, -1.4054e-01, -3.4238e-01,
         -1.1486e-01,  1.7757e-02,  1.6089e-01,  1.1305e-01,  2.0600e-01],
        [ 4.3876e-02, -2.4356e-01, -3.6768e-02, -1.6803e-01, -4.1378e-01,
         -1.0991e-01, -9.0277e-02,  2.0905e-01,  1.0021e-01,  2.7105e-01],
        [ 6.1471e-02, -1.9524e-01, -3.5281e-02, -2.0903e-01, -4.9177e-01,
         -1.3667e-01, -3.9006e-02,  1.3681e-01,  9.4053e-02,  2.9455e-01],
        [ 1.0323e-01, -2.0297e-01,  1.6410e-02, -1.5654e-01, -4.5607e-01,
         -9.2152e-02, -1.1012e-01,  1.2496e-01, -4.2502e-02,  9.5890e-02],
        [ 1.9453e-02, -3.0922e-01,  1.4043e-02, -3.3292e-01, -4.7440e-01,
         -1.0758e-01, -3.2999e-01,  1.1935e-02, -2.6625e-02,  2.0814e-01],
        [ 5.0278e-02, -2.1636e-01,  9.4281e-03, -1.0958e-01, -2.8043e-01,
         -1.3295e-01, -4.4020e-02,  1.7139e-01,  6.0562e-02,  2.0760e-01],
        [ 6.4749e-02, -2.5311e-01,  5.3786e-03, -2.6483e-01, -4.3790e-01,
         -1.3391e-01, -2.9853e-0

In [103]:
_,(lstm_new_out2,_) = test_lstm_new2(match_2)

In [104]:
lstm_new_out2 = lstm_new_out2.view(lstm_new_out2.size(1), lstm_new_out2.size(2))

In [105]:
lstm_new_out2.size()

torch.Size([32, 10])

Concat

In [108]:
concat_out = torch.cat((lstm_new_out1, lstm_new_out2), dim = 1)

In [109]:
concat_out.size()

torch.Size([32, 20])

In [110]:
test_linear = nn.Linear(2*hidden_dim2, 2)

In [111]:
final_result = test_linear(concat_out)

In [112]:
final_result

tensor([[-0.1265, -0.1354],
        [-0.1529, -0.1801],
        [-0.1598, -0.1770],
        [-0.1626, -0.1419],
        [-0.2798, -0.2700],
        [-0.1413, -0.1438],
        [-0.2489, -0.2500],
        [-0.1943, -0.2145],
        [-0.2543, -0.2329],
        [-0.2819, -0.2134],
        [-0.0779, -0.1065],
        [-0.3218, -0.2998],
        [-0.0190, -0.0378],
        [-0.0767, -0.0699],
        [-0.1187, -0.1352],
        [-0.2468, -0.2202],
        [-0.1324, -0.1801],
        [-0.1908, -0.1914],
        [-0.1820, -0.1946],
        [-0.1330, -0.1527],
        [-0.2085, -0.2137],
        [-0.1024, -0.0977],
        [-0.2568, -0.2194],
        [-0.1705, -0.1670],
        [-0.1559, -0.1977],
        [-0.1661, -0.1780],
        [-0.2008, -0.1846],
        [-0.1797, -0.2089],
        [-0.2185, -0.2400],
        [-0.1688, -0.1802],
        [-0.2803, -0.2481],
        [-0.2228, -0.1721]], grad_fn=<AddmmBackward>)

In [119]:
tag_scores = F.softmax(final_result, dim = 1)

In [120]:
tag_scores

tensor([[0.5022, 0.4978],
        [0.5068, 0.4932],
        [0.5043, 0.4957],
        [0.4948, 0.5052],
        [0.4976, 0.5024],
        [0.5006, 0.4994],
        [0.5003, 0.4997],
        [0.5051, 0.4949],
        [0.4946, 0.5054],
        [0.4829, 0.5171],
        [0.5071, 0.4929],
        [0.4945, 0.5055],
        [0.5047, 0.4953],
        [0.4983, 0.5017],
        [0.5041, 0.4959],
        [0.4934, 0.5066],
        [0.5119, 0.4881],
        [0.5002, 0.4998],
        [0.5032, 0.4968],
        [0.5049, 0.4951],
        [0.5013, 0.4987],
        [0.4988, 0.5012],
        [0.4906, 0.5094],
        [0.4991, 0.5009],
        [0.5105, 0.4895],
        [0.5030, 0.4970],
        [0.4959, 0.5041],
        [0.5073, 0.4927],
        [0.5054, 0.4946],
        [0.5029, 0.4971],
        [0.4920, 0.5080],
        [0.4873, 0.5127]], grad_fn=<SoftmaxBackward>)

In [146]:
predict_result = tag_scores[:, -1]

In [147]:
predict_result

tensor([0.4978, 0.4932, 0.4957, 0.5052, 0.5024, 0.4994, 0.4997, 0.4949, 0.5054,
        0.5171, 0.4929, 0.5055, 0.4953, 0.5017, 0.4959, 0.5066, 0.4881, 0.4998,
        0.4968, 0.4951, 0.4987, 0.5012, 0.5094, 0.5009, 0.4895, 0.4970, 0.5041,
        0.4927, 0.4946, 0.4971, 0.5080, 0.5127], grad_fn=<SelectBackward>)

In [152]:
test_loss = loss_fn(predict_result, batch_sample[2])

In [153]:
test_loss

tensor(0.7864, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [157]:
batch_value = batch_sample[2].numpy()
batch_value

array([0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1.],
      dtype=float32)

# LSTM

In [70]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [71]:
embedding_dim = vocab_size
embedding_dim

47549

In [72]:
hidden_dim = 50

In [73]:
match_size = 30

In [74]:
hidden_dim2 = 10

In [114]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, match_size, hidden_dim2):
        super(LSTMTagger, self).__init__()
        
        self.embedding = nn.Embedding(embedding_dim, 100)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32)) 

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm1 = nn.LSTM(input_size = 100, hidden_size = hidden_dim, batch_first = True)
        self.lstm2 = nn.LSTM(input_size = 100, hidden_size = hidden_dim, batch_first = True)
        
        # Matching layer
        self.matchpara1 = nn.Parameter(torch.rand(match_size, hidden_dim))
        self.matchpara2 = nn.Parameter(torch.rand(match_size, hidden_dim))
        self.cosSim = nn.CosineSimilarity(dim=1)
        
        # Aggregation layer
        self.lstm_new1 = nn.LSTM(input_size = match_size, hidden_size = hidden_dim2, batch_first=True)
        self.lstm_new2 = nn.LSTM(input_size = match_size, hidden_size = hidden_dim2, batch_first=True)
        
        # Prediction layer
        self.predlinear = nn.Linear(2*hidden_dim2, 2)
        
    def forward(self, input1, input2):
        #print(input1.size())
        embeds1 = self.embedding(input1)
        #print(embeds1.size())
        lstm_out1, _ = self.lstm1(embeds1)
        embeds2 = self.embedding(input2)
        lstm_out2, _ = self.lstm2(embeds2)
        
        # matching
        match_1 = self.match_cal(lstm_out1, lstm_out2, self.matchpara1)
        match_2 = self.match_cal(lstm_out2, lstm_out1, self.matchpara2)
        
        # aggregating
        _,(lstm_new_out1,_) = self.lstm_new1(match_1)
        _,(lstm_new_out2,_) = self.lstm_new2(match_2)
        
        # predicting
        lstm_new_out1 = lstm_new_out1.view(lstm_new_out1.size(1), lstm_new_out1.size(2))
        lstm_new_out2 = lstm_new_out2.view(lstm_new_out2.size(1), lstm_new_out2.size(2))
        concat_out = torch.cat((lstm_new_out1, lstm_new_out2), dim = 1)
        final_result = self.predlinear(concat_out)
        tag_scores = F.softmax(final_result, dim=1)
        
        return tag_scores
    
    def match_cal(self, lstm_out1, lstm_out2, matchpara):
        match_out_all = []
        for b in range(lstm_out1.size(0)):
            match_out_lst = []
            lstm_out1_batch1 = lstm_out1[b,:,:]
            #lstm_out2_batch1_end = lstm_out2[b, -1, :]
            for seq in range(lstm_out1_batch1.size(0)):
                batch1_t = lstm_out1_batch1[seq,:]
                #batch2_t = lstm_out2_batch1_end
                batch2_t = self.matching_method(batch1_t, lstm_out2[b,:,:])
                mod_batch1_t = torch.mul(matchpara, batch1_t)
                mod_batch2_t = torch.mul(matchpara, batch2_t)
                match_out_t = self.cosSim(mod_batch1_t, mod_batch2_t)
                match_out_lst.append(match_out_t)
            match_out = torch.stack(match_out_lst)
            match_out_all.append(match_out)
        return torch.stack(match_out_all)
    
    def matching_method(self, batch1_t, lstm_out2_batch1):
        prep_batch1_t = batch1_t.repeat(lstm_out2_batch1.size(0),1)
        weight = self.cosSim(prep_batch1_t, lstm_out2_batch1).detach().numpy()
        max_weight = np.argmax(weight)
        return lstm_out2_batch1[max_weight,:]

### Train

In [141]:
loss_input = torch.randn(3, requires_grad=True)
loss_target = torch.empty(3).random_(2)
loss_output = loss_fn(input, target)

In [142]:
(loss_input, loss_target)

(tensor([ 1.7795, -2.2021, -0.2727], requires_grad=True), tensor([0., 0., 0.]))

In [143]:
loss_output

tensor(1.3420, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [76]:
from sklearn.metrics import f1_score

In [115]:
LSTMmodel = LSTMTagger(embedding_dim, hidden_dim, match_size, hidden_dim2)

In [116]:
loss_fn = nn.BCEWithLogitsLoss(reduction='sum')
optimizer = optim.Adam(LSTMmodel.parameters(), lr=1e-3)

In [117]:
n_epochs = 1

In [118]:
for epoch in range(n_epochs):  
    # Train
    LSTMmodel.train()
    avg_loss = 0. 
    batch_num = 1
    for ques1, ques2, label in train_dataloader:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        LSTMmodel.zero_grad()
        # Step 2. Run our forward pass.
        tag_scores = LSTMmodel(ques1, ques2)
        predict_scores = tag_scores[:,-1]

        # Step 3. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        
        loss = loss_fn(predict_scores, label)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item() / len(train_dataloader)
        if batch_num % 20 == 1:
            print('Batch {} \t loss={:4f}'.format(batch_num, avg_loss))
        batch_num += 1
        
    # Evaluation
    LSTMmodel.eval()
    
    batch_num = 1
    avg_metric = 0.
    for ques1, ques2, label in validation_dataloader:
        with torch.no_grad():
            tag_scores = LSTMmodel(ques1, ques2)
            
        predict_label = np.argmax(tag_scores.detach().numpy(), axis=1).flatten()
        avg_metric += f1_score(predict_label, label.numpy()) / len(validation_dataloader)
        if batch_num % 20 == 1:
            print('Batch {} \t metric={:4f}'.format(batch_num, avg_metric))
        batch_num += 1
        
    # Output
    print('Epoch {}/{} \t loss={:.4f} \t metric={:.4f}'.format(
                epoch + 1, n_epochs, avg_loss, avg_metric))
        

Batch 1 	 loss=0.010063
Batch 21 	 loss=0.210014
Batch 41 	 loss=0.396859
Batch 61 	 loss=0.577218
Batch 81 	 loss=0.755639
Batch 101 	 loss=0.933963
Batch 121 	 loss=1.112021
Batch 141 	 loss=1.289865
Batch 161 	 loss=1.467698
Batch 181 	 loss=1.645412
Batch 201 	 loss=1.823032
Batch 221 	 loss=2.000650
Batch 241 	 loss=2.178279
Batch 261 	 loss=2.355894
Batch 281 	 loss=2.533497
Batch 301 	 loss=2.711055
Batch 321 	 loss=2.888586
Batch 341 	 loss=3.066128
Batch 361 	 loss=3.243645
Batch 381 	 loss=3.421173
Batch 401 	 loss=3.598677
Batch 421 	 loss=3.776189
Batch 441 	 loss=3.953694
Batch 461 	 loss=4.131199
Batch 481 	 loss=4.308706
Batch 501 	 loss=4.486209
Batch 521 	 loss=4.663712
Batch 541 	 loss=4.841208
Batch 561 	 loss=5.018696
Batch 581 	 loss=5.196179
Batch 601 	 loss=5.373668
Batch 621 	 loss=5.551143
Batch 641 	 loss=5.728620
Batch 661 	 loss=5.906102
Batch 681 	 loss=6.083581
Batch 701 	 loss=6.261057
Batch 721 	 loss=6.438532
Batch 741 	 loss=6.616010
Batch 761 	 loss=6